### 6. Setup

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7982
WEBHOOK_BASE = ""
ADMIN_URL = "http://the_wallet-agent:7981"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_operation_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_operation_credential


### 7. Go to hospital notebook to issue credential

### 8. Get the Record

In [4]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'auto_issue': False, 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.5', 'role': 'holder', 'updated_at': '2023-01-06 05:13:25.585488Z', 'credential_definition_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:87:default', 'credential_offer': {'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.5', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:87:default', 'key_correctness_proof': {'c': '58927600743692660555416866094847049558699470913006119651249499662498926774847', 'xz_cap': '1203692133746237943782214988568982890119930416088555910329486170197214803379739371706380871953184164484113717100096821911424407425567443251903696844864787010396722208992411938713728660306544176316552003946032682412398249666973888165020835196223760756313082953699237640920496223211785320479960545905019360649666887424558818194844649617681923747032268390941806484042596014803328291549627512859062683821648867543214262561780364572394627558308693140000815327792129135597777031807374499847306225043659802297705144151464315

In [5]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange e67f9ded-fdaf-4217-94e1-8af7ec525a29, role: holder, state: offer_received
Being offered: [{'name': 'type', 'value': 'drug_A'}, {'name': 'dose', 'value': '1'}, {'name': 'speed', 'value': '1'}, {'name': 'volt', 'value': '1'}]


### 9. Send request for record

In [6]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange e67f9ded-fdaf-4217-94e1-8af7ec525a29, role: holder, state: request_sent


### 10. Store Credential


In [7]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "Operation n")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange e67f9ded-fdaf-4217-94e1-8af7ec525a29, role: holder, state: credential_acked


You can get  credential by your credential id

### **Terminate**

In [8]:
response = await agent_controller.terminate()
print(response)

None
